In [5]:
import os
# from tqdm.notebook import tqdm
import ipywidgets as widgets
from ipywidgets import Video, Button, VBox, HBox
from IPython.display import display, clear_output, HTML
from ipywidgets.embed import embed_minimal_html
from tensorflow.keras.models import load_model
from ultralytics import YOLO

import violence_detection 


In [4]:
folder_path = './project_databases/movies'

def get_all_video_names():
    
    # List of video paths
    video_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(('.mp4', '.avi', '.mkv', '.webm'))]
    video_names = [os.path.splitext(os.path.basename(path))[0] for path in video_paths]
    sorted_video_names = sorted([f"{video}" for index, video in enumerate(video_names)])
    return sorted_video_names

def display_video(video_path):
    video_width = 640
    video_height = 480
    html_code = f"""
            <video width="{video_width}" height="{video_height}" controls loop>
                <source src="{video_path}" type="video/mp4">
                Your browser does not support the video tag.
            </video>
            """
    # Display the HTML code
    display(HTML(html_code))

def display_header():
    header_html = f"""
        <div style="padding: 20px;">
            <h1 style="color: black; font-family: 'Times New Roman', serif; font-size: 30px; font-weight: bold;">Violence detection in Videos</h1>
        </div>
        """
    display(HTML(header_html))
    

def display_violent_out(label, pred_prob):
    
    color, header = ('red', "Violent Content!") if label=="Violence" else ('green', "No Violent Content")
    confidence =  "{:.2f}".format(pred_prob*100) 
        
    html_code  = f"""
                <div>
                <p style="font-size: 22px; font-family: 'Arial', sans-serif; font-weight: bold; color: {color}; margin: 0; padding: 0;">{header}</p>
                <p style="font-size: 20px; font-family: 'Arial', sans-serif; color: black; margin: 0; padding: 0;">Confidence: {confidence}%</p>
                </div>
                """
    display(HTML(html_code))

def display_weapons_out(weapons):
    
    color, header = ('red', "Detect Weapons!") if weapons else ('green', "No weapons found")
        
    html_code  = f"""
                <div>
                <p style="font-size: 22px; font-family: 'Arial', sans-serif; font-weight: bold; color: {color}; margin: 0; padding: 0;">{header}</p>
                </div>
                """
    display(HTML(html_code))

def display_all(label, pred_prob, weapons, folowing):
    
    color, header = ('red', "Detect violent content!") if label=="Violence" or (weapons and folowing) else ('green', "No violent content")

    violence_header = "violent behavor" if label=="Violence" else ""
    weapons_header = "weapons found" if (weapons and folowing) else ""
    both = ", " if label=="Violence" and (weapons and folowing) else ""

        
    html_code  = f"""
                <div>
                <p style="font-size: 22px; font-family: 'Arial', sans-serif; font-weight: bold; color: {color}; margin: 0; padding: 0;">{header}</p>
                <p style="font-size: 16px; font-family: 'Arial', sans-serif; color: black; margin: 0; padding: 0;">{violence_header}{both}{weapons_header}</p>
                </div>
                """
    display(HTML(html_code))
    
# Initial options
options = get_all_video_names()
filtered_options = options

# Dropdown widget to select a video
video_dropdown = widgets.Dropdown(
    options=filtered_options,
    description='Select:',
    style={'description_width': 'initial'},
    layout={'width': '224px'}
)

# Text input widget for filtering
text_input = widgets.Text(
    value='',
    placeholder='E.g., "V", "N" or "W"',
    description='Filter:',
    style={'description_width': 'initial'},
    layout={'width': '224px'}
)

# # Video widget to display the selected video
# video_widget = widgets.Video()

# Output widget to handle widget updates
output_widget = widgets.Output()

# Buttons
# play_button = widgets.Button(description="Play Video")
detect_violence_button = widgets.Button(description="Detect Violence")
detect_weapons_button = widgets.Button(description="Detect Weapons")
detect_button = widgets.Button(description="Detect")
clear_button = Button(description='Clear')



# Variable to store the selected video path
selected_video_path = None

# Load violence detection model
LRCN = load_model('LRCN.h5')

# Load weapons detection model
model_path = os.path.join('.', 'runs', 'detect', 'train2', 'weights', 'best.pt')
weapons_detection_model = YOLO(model_path)  # load a custom model

def update_dropdown_options(change):

    global selected_video_path
    
    search_term = change.new.lower()
    if search_term:
        starting_letter = search_term[0]
        video_dropdown.options = [option for option in options if option.lower().startswith(starting_letter.lower())]
    else:
        video_dropdown.options = options
    
# Function to update the displayed video
def update_video(change):
    
    global selected_video_path
    
    selected_option = change.new
    selected_video_path = f"{folder_path}/{selected_option.replace(' ', '_')}.mp4"  # Replace with your actual video path
    
    with output_widget:
        output_widget.clear_output()
        display_video(selected_video_path)


def detect_violence_on_video(button_click):
    
    global selected_video_path
    if selected_video_path is not None:
        
        pred_class, pred_prob = violence_detection.detect_violence(LRCN, selected_video_path)
        
        with output_widget:
            output_widget.clear_output(wait=True)
            display_violent_out(pred_class, pred_prob)
            display_video(selected_video_path)


# Function to handle button click event for...
def detect_weapons_on_video(button_click):
    
    global selected_video_path
    if selected_video_path is not None:
        
        weapons, movie_with_boxes, folowing = violence_detection.detect_weapons(weapons_detection_model, 0.65, selected_video_path, True)
        
        with output_widget:
            output_widget.clear_output(wait=True)
            display_weapons_out(weapons)
            display_video(movie_with_boxes)
                
def detect(button_click):
    
    global selected_video_path
    
    if selected_video_path is not None:
        with output_widget:
            violence_Y_N, confidence, weapons_found, movie_with_boxes, folowing = violence_detection.detect(LRCN, weapons_detection_model, 0.65, selected_video_path)
            output_widget.clear_output(wait=True)
            display_all(violence_Y_N, confidence, weapons_found, folowing)
            vid_to_display = movie_with_boxes if folowing else selected_video_path
            display_video(vid_to_display)
    
# Attach the update function to the text input widget's observe method
text_input.observe(update_dropdown_options, names='value')

# Attach the update function to the dropdown's change event
video_dropdown.observe(update_video, names='value')

def clear_output_func(button_click):
    
    with output_widget:
        output_widget.clear_output()


detect_violence_button.on_click(detect_violence_on_video)
detect_weapons_button.on_click(detect_weapons_on_video)
detect_button.on_click(detect)
clear_button.on_click(clear_output_func)

# Display widgets
display_header()
buttons_hbox1 = HBox([text_input, video_dropdown])
buttons_hbox2 = HBox([detect_button, detect_violence_button, detect_weapons_button, clear_button])
display(buttons_hbox1, buttons_hbox2, output_widget)

# Embed HTML for the widgets (optional)
embed_minimal_html('export.html', views=[video_dropdown, detect_violence_button, clear_button, output_widget], title='Video Player')



Output()